In [1]:
import random

# Coroutines
Coroutines are generalizations of subroutines. They are used for cooperative multitasking where a process voluntarily yield (give away) control periodically or when idle in order to enable multiple applications to be run simultaneously. The difference between coroutine and subroutine is :  

+ **Unlike subroutines**, coroutines have many entry points for suspending and resuming execution. Coroutine can suspend its execution and transfer control to other coroutine and can resume again execution from the point it left off.


+ **Unlike subroutines**, there is no main function to call coroutines in a particular order and coordinate the results. Coroutines are cooperative that means they link together to form a pipeline. One coroutine may consume input data and send it to other that process it. Finally, there may be a coroutine to display the result.


Now you might be thinking how coroutine is different from threads, both seem to do the same job.
In the case of threads, it’s an operating system (or run time environment) that switches between threads according to the scheduler. While in the case of a coroutine, it’s the programmer and programming language which decides when to switch coroutines. Coroutines work cooperatively multitask by suspending and resuming at set points by the programmer.

***Nice Read:*** https://www.dabeaz.com/coroutines/Coroutines.pdf

## Difference b/n Courutines/Generators

You're correct that both generators and generator-based coroutines can receive data through the send() method, which can blur the distinction between the two. However, there are key conceptual and practical differences that separate them:

### 1. *Intended Use Case*
   - *Generators*: Primarily designed to produce a sequence of values, often used in iteration. Their primary focus is on yielding data outward, although they can also receive data using send().
   - *Generator-Based Coroutines*: Intended for cooperative multitasking, where they manage tasks, handle events, or maintain state across multiple send() calls. The focus is on controlling the flow of execution, often based on external inputs.

### 2. *Flow Control*
   - *Generators*: Typically follow a linear flow, where the generator produces a sequence of values until it is exhausted. The send() method allows external data to influence the next value generated, but the primary flow is still to produce a sequence.
   - *Generator-Based Coroutines*: The flow is more event-driven or state-driven. They wait for data (or events) to be sent to them, react to that data, and may or may not produce a value in response. The emphasis is on reacting to inputs rather than simply producing outputs.

### 3. *Return vs. Yield*
   - *Generators*: The goal is usually to return values over time, yielding multiple values during its execution.
   - *Generator-Based Coroutines*: The goal is typically to manage state or handle inputs without necessarily producing a series of outputs. The coroutine may yield for the purpose of pausing execution rather than for producing a value.

### 4. *Lifecycle and Termination*
   - *Generators*: They generally run until they exhaust their sequence of values, at which point they raise StopIteration.
   - *Generator-Based Coroutines*: They often run indefinitely, handling inputs until they are explicitly terminated (e.g., via close()), making them suitable for long-running tasks or event loops.

### 5. *Conceptual Focus*
   - *Generators*: Conceptually focused on generating data in a lazy fashion.
   - *Generator-Based Coroutines*: Conceptually focused on managing tasks, handling data over time, and maintaining a responsive state.

### Example of Differences:

- *Generator Example*:
```
  python
  def simple_generator():
      for i in range(5):
          x = yield i  # Yield a value, optionally receive one via send()
          print(f"Received from send: {x}")

  gen = simple_generator()
  print(next(gen))  # Outputs: 0
  gen.send(10)  # Outputs: Received from send: 10, then yields 1
 ```

- *Coroutine Example*:
```
  python
  def simple_coroutine():
      print("Coroutine started")
      while True:
          value = yield  # Wait for a value to be sent
          print(f"Received: {value}")
          if value == "exit":
              break

  coro = simple_coroutine()
  next(coro)  # Start the coroutine, "Coroutine started" is printed
  coro.send("Hello")  # Outputs: Received: Hello
  coro.send("exit")  # Outputs: Received: exit, then exits
 ```

In this example, the generator is primarily focused on producing values (even if influenced by send()), while the coroutine is more focused on managing the flow of execution based on the data it receives.

## Consuming Data

Generators produce data for iteration while coroutines can also consume data.
In Python 2.5, a slight modification to the yield statement was introduced, now yield can also be used as an expression. For example on the right side of the assignment –
```
line = (yield)
```

whatever value we send to coroutine is captured and returned by (yield) expression.

A value can be sent to the coroutine by send() method.

In [4]:
def greet_me():
    print("Introduce Yourself:")
    while True:
        name = (yield)
        print(f"Hello {name}")

In [10]:
# Creating Coroutine
func = greet_me()

In [14]:
# This will start execution of coroutine
# next(...) will also work
# Generators do not have __next__ method
func.__next__()

Hello None


In [8]:
func.send("Ayush")

Hello Ayush


In [9]:
func.send("Ayush Thada")

Hello Ayush Thada


In [16]:
func

<generator object greet_me at 0x7ef27b98e180>

## Closing Coroutines

In [18]:
def greet_me():
    print("Introduce Yourself:")
    try:
      while True:
          name = (yield)
          print(f"Hello {name}")
    except GeneratorExit:
      print("Clsoing Coroutine")

func = greet_me()
next(func)
func.send("Ayush")
func.send("Ayush Thada")
func.close()

Introduce Yourself:
Hello Ayush
Hello Ayush Thada
Clsoing Coroutine


#### Courioutine Chains to make a Pipeline

In [22]:
def producer(sentence, next_coroutine):
    '''
    Producer which just split strings and
    feed it to pattern_filter coroutine
    '''
    tokens = sentence.split(" ")
    for token in tokens:
        next_coroutine.send(token)
    next_coroutine.close()

In [23]:
def pattern_filter(pattern="ing", next_coroutine=None):
    '''
    Search for pattern in received token
    and if pattern got matched, send it to
    print_token() coroutine for printing
    '''
    print("Searching for {}".format(pattern))
    try:
        while True:
            token = (yield)
            if pattern in token:
                next_coroutine.send(token)
    except GeneratorExit:
        print("Done with filtering!!")
        next_coroutine.close()

In [24]:
def print_token():
    '''
    Act as a sink, simply print the
    received tokens
    '''
    print("I'm sink, i'll print tokens")
    try:
        while True:
            token = (yield)
            print(token)
    except GeneratorExit:
        print("Done with printing!")

In [25]:
pt = print_token()
pt.__next__()
pf = pattern_filter(next_coroutine = pt)
pf.__next__()

I'm sink, i'll print tokens
Searching for ing


In [26]:
sentence = "Bob is running behind a fast moving car"
producer(sentence, pf)

running
moving
Done with filtering!!
Done with printing!
